In [1]:
import sys
sys.path.append('/Users/efraflores/Desktop/hub/cornershop/venv/lib/python3.9/site-packages')

# Days between orders

In [2]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Corner/Brands/Requests'
FILE_BASE_NAME = 'dbo'

In [3]:
import re
import os

FILE_NAME_LIST = [x for x in os.listdir(BASE_DIR) if re.search(fr'{FILE_BASE_NAME}_\d+\.csv',x)!=None]
print(sorted(FILE_NAME_LIST))

['dbo_2003.csv', 'dbo_2004.csv', 'dbo_2101.csv', 'dbo_2102.csv']


## Functions

### Timing and tone

In [4]:
import time
import numpy as np
from IPython.lib.display import Audio

start = time.time()
def time_exp(x):
    minutes, seconds = np.floor(x/60), 60*(x/60-np.floor(x/60))
    print(f"{'{:.0f}'.format(minutes)} minutos con {'{:.2f}'.format(seconds)} segundos")
    
def tono(a=1000, b=700, play_time_seconds=1, framerate=4410):
    t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)*np.pi
    return Audio(np.sin(a*t) + np.sin(b*t), rate=framerate, autoplay=True)

### Percentiles

In [5]:
def perc80(x):
    return np.percentile(x,80)

### Files in chunks

In [6]:
import pandas as pd

def files_chunks(base_dir,file_list,chunksize,prefunction,**kwargs):
    df = pd.DataFrame()
    for file in file_list:
        chunks = pd.read_csv(os.path.join(base_dir,file),
                             #sep='\t', encoding='utf-16',
                             chunksize=chunksize)
        for i,chunk in enumerate(chunks):
            df = df.append(prefunction(chunk,**kwargs),ignore_index=False)
    return df

### Prefiltering

In [7]:
def pre_filtering(data,index_cols,dict_values):
    df = data.groupby(index_cols).agg(dict_values)
    df.columns = ['_'.join([x for x in col]) if 
                  not isinstance(df.columns[0],str) #First element is not a string
                  else col for col in df.columns]
    return df

## Import

In [8]:
df = files_chunks(BASE_DIR,FILE_NAME_LIST,
                  chunksize=2e6,prefunction=pre_filtering,
                  index_cols=['year','month','city'],
                  dict_values={'user_id':'nunique',
                               'n_order':['count','mean','median',perc80,'max'],
                               'requested':sum,'found':sum,'replaced':sum,'sales':sum,
                               'days_elapsed':['mean','median',perc80,'max']})

print(len(df))
display(df.sample())

171


,,,user_id_nunique,n_order_count,n_order_mean,n_order_median,n_order_perc80,n_order_max,requested_sum,found_sum,replaced_sum,sales_sum,days_elapsed_mean,days_elapsed_median,days_elapsed_perc80,days_elapsed_max
year,month,city,,,,,,,,,,,,,,
2021,1,Guadalajara,3707,8987,4.199177,3,6.0,41,148770,126759,13199,10920717.93,6.506543,5.516798,9.793345,30.110959


In [9]:
df = files_chunks(BASE_DIR,FILE_NAME_LIST,chunksize=3e6,prefunction=lambda x: x)
print(len(df))
display(df.sample())

3655539


,year,month,week,city,user_id,n_order,requested,found,replaced,sales,days_elapsed
585995,2020,12,50,Ciudad de México,249697,21,2,2,0,625.0,2.161667


In [24]:
export = df.pivot_table(index=['year','month'],
                        columns='city',
                        values='days_elapsed',
                        aggfunc = ['mean','median',perc80,'max'],
                        fill_value=0,
                        margins=True)
#export.columns = [x[0] for x in export.columns]
export.head(7)

mean                                                      \
city       Aguascalientes    Cancún  Chihuahua Ciudad Juárez Ciudad de México   
year month                                                                      
2020 7           0.000000  5.431158   6.108671           0.0         5.706466   
     8           0.000000  7.493472   8.383169           0.0         8.176991   
     9           0.000000  9.302581  10.107243           0.0         9.503933   
     10          0.000000  5.857886   5.789885           0.0         5.687814   
     11          7.614949  7.989672   8.016093           0.0         8.204922   
     12         10.061005  9.149710   9.472457           0.0         9.125537   
2021 1           6.545063  5.332455   6.093701           0.0         5.304380   

                                                                  ...  \
city       Cuernavaca Culiacán Guadalajara Hermosillo       León  ...   
year month                                                        ...   
2020 7            0.0      0.0    6.767175        0.0   5.830176  ...   
     8            0.0      0.0   10.342936        0.0   8.797591  ...   
     9            0.0      0.0   12.560450        0.0  10.260715  ...   
     10           0.0      0.0    6.844683        0.0   6.160650  ...   
     11           0.0      0.0   10.135587        0.0   9.324736  ...   
     12           0.0      0.0   12.788306        0.0  10.366033  ...   
2021 1            0.0      0.0    6.506543        0.0   5.835316  ...   

                max                                                      \
city       Mexicali  Monterrey Morelia     Mérida     Puebla  Querétaro   
year month                                                                
2020 7          0.0  30.261842     0.0  29.776157  29.312046  29.309465   
     8          0.0  61.227797     0.0  61.042178  58.280665  59.029502   
     9          0.0  90.956844     0.0  85.856158  91.425089  89.111402   
     10         0.0  30.194918     0.0  30.144414  29.847083  29.856940   
     11         0.0  60.368770     0.0  58.352958  58.129296  58.941399   
     12         0.0  89.197724     0.0  87.943818  87.860490  91.168652   
2021 1          0.0  28.831036     0.0  29.051177  29.972232  28.868979   

                                                          
city       San Luis Potosí Tijuana     Toluca        All  
year month                                                
2020 7            0.000000     0.0  27.327392  30.261842  
     8            0.000000     0.0  57.867341  61.227797  
     9            0.000000     0.0  84.086148  91.425089  
     10           0.000000     0.0  29.105082  30.450684  
     11          26.237310     0.0  55.095412  60.368770  
     12          48.014588     0.0  86.906710  91.168652  
2021 1           21.048058     0.0  29.149753  30.147343  

[7 rows x 80 columns]

In [25]:
export = export.melt(ignore_index=False).rename(columns={None:'func'})
export.head(7)

NaN            city      value
year month                                 
2020 7      mean  Aguascalientes   0.000000
     8      mean  Aguascalientes   0.000000
     9      mean  Aguascalientes   0.000000
     10     mean  Aguascalientes   0.000000
     11     mean  Aguascalientes   7.614949
     12     mean  Aguascalientes  10.061005
2021 1      mean  Aguascalientes   6.545063

## Export

In [31]:
#df.to_csv(os.path.join(BASE_DIR,FILE_BASE_NAME+'.csv'),sep='\t',encoding='utf-16')
export.to_csv(os.path.join(BASE_DIR,FILE_BASE_NAME+'.csv'),sep='\t',encoding='utf-16')

## End

In [12]:
time_exp(time.time()-start)
tono()

0 minutos con 11.78 segundos
